In [113]:
%matplotlib inline
import json
import oandapyV20
import oandapyV20.endpoints.instruments as instruments

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from oandapyV20.contrib.factories import InstrumentsCandlesFactory
from myauth import myauth

In [25]:
accountID, access_token = myauth.myAuth()
client = oandapyV20.API(access_token=access_token)

In [134]:
instrument = "EUR_USD"
params = {
    "from": "2019-01-16T22:00:00Z",
    "to":   "2019-01-17T22:00:00Z",
    "granularity": "S15",
    "includeFirst": True,
#     "count": 2,
}

r = instruments.InstrumentsCandles(instrument=instrument, params=params)
response = client.request(r)
print("Request: {}  #candles received: {}".format(r, len(r.response.get('candles'))))
print(json.dumps(response, indent=2))

In [136]:
def cnv(r, h):
    # get all candles from the response and write them as a record to the filehandle h
    for candle in r.get('candles'):
        ctime = candle.get('time')[0:19]
        try:
            rec = "{time},{complete},{o},{h},{l},{c},{v}".format(
                time=ctime,
                complete=candle['complete'],
                o=candle['mid']['o'],
                h=candle['mid']['h'],
                l=candle['mid']['l'],
                c=candle['mid']['c'],
                v=candle['volume'],
            )
        except Exception as e:
            print(e, r)
        else:
            h.write(rec+"\n")

In [137]:
datafile = "/tmp/{}.{}.out".format(instrument, params['granularity'])
with open(datafile, "w") as O:
    n = 0
    for r in InstrumentsCandlesFactory(instrument=instrument, params=params):
        rv = client.request(r)
        cnt = len(r.response.get('candles'))
        print("REQUEST: {} {} {}, received: {}".format(r, r.__class__.__name__, r.params, cnt))
        n += cnt
        cnv(r.response, O)
    print("Check the datafile: {} under /tmp!, it contains {} records".format(datafile, n))

REQUEST: v3/instruments/EUR_USD/candles InstrumentsCandles {'granularity': 'S15', 'includeFirst': True, 'from': '2019-01-16T22:00:00Z', 'to': '2019-01-17T00:05:00Z'}, received: 231
REQUEST: v3/instruments/EUR_USD/candles InstrumentsCandles {'granularity': 'S15', 'includeFirst': True, 'from': '2019-01-17T00:05:00Z', 'to': '2019-01-17T02:10:00Z'}, received: 301
REQUEST: v3/instruments/EUR_USD/candles InstrumentsCandles {'granularity': 'S15', 'includeFirst': True, 'from': '2019-01-17T02:10:00Z', 'to': '2019-01-17T04:15:00Z'}, received: 300
REQUEST: v3/instruments/EUR_USD/candles InstrumentsCandles {'granularity': 'S15', 'includeFirst': True, 'from': '2019-01-17T04:15:00Z', 'to': '2019-01-17T06:20:00Z'}, received: 303
REQUEST: v3/instruments/EUR_USD/candles InstrumentsCandles {'granularity': 'S15', 'includeFirst': True, 'from': '2019-01-17T06:20:00Z', 'to': '2019-01-17T08:25:00Z'}, received: 438
REQUEST: v3/instruments/EUR_USD/candles InstrumentsCandles {'granularity': 'S15', 'includeFirst

In [140]:
df = pd.read_csv(filepath_or_buffer='/tmp/EUR_USD.S15.out', names=['time','complete','o','h','l','c','v'])

df['time'] = pd.to_datetime(df['time'])
df['oc'] = df['c'] - df['o']
df['hl'] = df['h'] - df['l']

In [36]:
def pcaWeights(cov, riskDist=None, riskTarget=1.):
    eVal, eVec = np.linalg.eigh(cov)
    indices = eVal.argsort()[::-1]
    eVal, eVec = eVal[indices], eVec[:,indices]
    if riskDist is None:
        riskDist = np.zeros(cov.shape[0])
        riskDist[-1] = 1
    loads = riskTarget * (riskDist/eVal)
    wghts = np.dot(eVec, np.reshape(loads, (-1, 1)))
    return wghts